In [5]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("omkargurav/face-mask-dataset")

print("Path to dataset files:", path)

Path to dataset files: /home/azureuser/.cache/kagglehub/datasets/omkargurav/face-mask-dataset/versions/1


In [38]:
import pandas as pd
import numpy as np
import cv2
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from tensorflow.keras.applications import ResNet50
from keras.models import Sequential
from keras.layers import Dense, Flatten, GlobalAveragePooling2D
from sklearn.metrics import confusion_matrix

In [7]:
classes=['with_mask','without_mask']

In [8]:
data=[]
for catgory in classes:
    file_path=os.path.join(path,"data/"+catgory)
    for img in os.listdir(file_path):
        image=cv2.imread(os.path.join(file_path,img))
        image=cv2.resize(image,(224,224))
        data.append([image,catgory])

In [9]:
 random.shuffle(data)

In [10]:
X=[]
y=[]
for features,labels in data:
    X.append(features)
    y.append(labels)

In [11]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()

y=le.fit_transform(y)

In [12]:
X=np.array(X)
y=np.array(y)

In [13]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

In [14]:
X_train=X_train/255
X_test=X_test/255

In [15]:
X_train.shape

(6042, 224, 224, 3)

In [16]:
y_train.shape

(6042,)

In [17]:
resnet=ResNet50(
    input_shape=(224,224,3),
    include_top=False,
    weights='imagenet'
)

2025-07-24 08:16:20.401087: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [18]:
resnet.summary()

Model: "resnet50"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 230, 230,  │          0 │ input_layer[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 112, 112,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 114, 114,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 56, 56,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 56, 56,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 56, 56,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 56, 56,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 56, 56,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_3_c

 Total params: 23,587,712 (89.98 MB)

 Trainable params: 23,534,592 (89.78 MB)

 Non-trainable params: 53,120 (207.50 KB)

In [19]:
model = Sequential()

In [20]:
model.add(resnet)

In [21]:
for layers in model.layers:
    layers.trainable=False

In [22]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,587,712 (89.98 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 23,587,712 (89.98 MB)

In [23]:
model.add(Flatten())
model.add(Dense(2,activation='sigmoid'))

In [24]:
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy'],
)

In [27]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 100352)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 2)              │       200,706 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,189,832 (92.28 MB)

 Trainable params: 200,706 (784.01 KB)

 Non-trainable params: 23,587,712 (89.98 MB)

 Optimizer params: 401,414 (1.53 MB)

In [28]:
model.fit(X_train,y_train,epochs=10,validation_data=(X_test,y_test))

Epoch 1/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 288s 2s/step - accuracy: 0.6823 - loss: 0.6273 - val_accuracy: 0.8008 - val_loss: 0.4653
Epoch 2/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 281s 1s/step - accuracy: 0.7627 - loss: 0.5342 - val_accuracy: 0.7823 - val_loss: 0.4685
Epoch 3/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 253s 1s/step - accuracy: 0.7821 - loss: 0.4815 - val_accuracy: 0.7280 - val_loss: 0.5579
Epoch 4/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 253s 1s/step - accuracy: 0.7796 - loss: 0.4764 - val_accuracy: 0.5586 - val_loss: 1.2259
Epoch 5/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 262s 1s/step - accuracy: 0.7888 - loss: 0.4778 - val_accuracy: 0.7975 - val_loss: 0.4603
Epoch 6/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 293s 2s/step - accuracy: 0.8161 - loss: 0.4167 - val_accuracy: 0.7809 - val_loss: 0.4741
Epoch 7/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 254s 1s/step - accuracy: 0.8028 - loss: 0.4775 - val_accuracy: 0.7968 - val_loss: 0.4717
Epoch 8/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 255s 1s/step - accuracy: 0.8009 - loss: 0.4551 - val_accu

In [39]:
preds= model.predict(X_test)

48/48 ━━━━━━━━━━━━━━━━━━━━ 51s 1s/step


[[501 251]
 [ 47 712]]


In [42]:
import pickle

pickle.dump(model, open('face_mask_detection_model.pkl', 'wb'))

In [43]:
from sklearn.metrics import classification_report, confusion_matrix 

In [44]:
print(classification_report(y_test, np.argmax(preds, axis=1), target_names=le.classes_))

              precision    recall  f1-score   support

   with_mask       0.91      0.67      0.77       752
without_mask       0.74      0.94      0.83       759

    accuracy                           0.80      1511
   macro avg       0.83      0.80      0.80      1511
weighted avg       0.83      0.80      0.80      1511



In [46]:
path="/home/azureuser/cloudfiles/code/images (1).jpeg"

img=image.load_img(path,target_size=(224,224))
img=image.img_to_array(img)
img=np.expand_dims(img,axis=0)
img=np.array(img)/255.0
pred=model.predict(img)
pred_class = np.argmax(pred, axis=1)
print("Predicted class:", le.inverse_transform(pred_class)[0])
print("Predicted probabilities:", pred[0])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
Predicted class: without_mask
Predicted probabilities: [0.5071237 0.917863 ]
